In [1]:
import calendar
import numpy as np
import pandas as pd
import h5py
import json
from itertools import compress
import gc
import sys
from hmmlearn import hmm
from random import random


In [12]:
# rng = np.random.default_rng(uncertainty_dict['synth_gen_seed'])
np.random.seed(42)
nSites = 15
mghmm_folder = 'calfews_mhmm_5112022/'

AnnualQ = pd.read_csv(mghmm_folder + "historical_annual_streamflow_all_locations.csv")
logAnnualQ = np.log(AnnualQ)
hmm_model = hmm.GMMHMM(n_components=2, n_iter=1000, covariance_type='full').fit(logAnnualQ)

mus = np.array(hmm_model.means_)
weights = np.array(hmm_model.weights_)
P = np.array(hmm_model.transmat_)
# print('mus', mus, 'weights', weights, 'P', P)
print(P)

[[0.7826125  0.2173875 ]
 [0.80750169 0.19249831]]


c:\Users\danli\.conda\envs\.venv_conda_calfews\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "
c:\Users\danli\.conda\envs\.venv_conda_calfews\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


In [23]:
start_year = 1996
historical_data = pd.read_csv(mghmm_folder + "historical_annual_streamflow_all_locations.csv")
historical_data['Year'] = 1906 + historical_data.index
historical_years = historical_data['Year']
historical_streamflows = historical_data.drop('Year', axis=1)

log_historical_streamflows = np.log(historical_streamflows)
historical_states = hmm_model.predict(log_historical_streamflows)

# Determine the state for the desired starting year (e.g., 1996)
start_state = historical_states[historical_years == start_year][0]
print(start_state)

0


In [28]:
import json

# Initialize the JSON structure
data = {
    "flow_input_type": [
        "observations", 
        "synthetic"
    ],
    
    "flow_input_source": {
        "observations": {
            "CDEC": "calfews_src/data/input/calfews_src-data-sim.csv", 
            "CDEC_wet_dry": "calfews_src/data/input/calfews_src-data-wet-dry.csv",
            "CDEC_short": "calfews_src/data/input/calfews_src-data.csv",
            "CDEC_1995_2023": "calfews_src/data/input/cord-sim_realtime.csv",
            "CDEC_1996_2024": "calfews_src/data/input/calfews_src-data_1.csv"
        },
        "synthetic": {}
    },
    
    "export_series": {
        "observations": {
            "CDEC": "calfews_src-data-CDEC_19052016",
            "CDEC_wet_dry": "calfews_src-data-CDEC_wet_dry",
            "CDEC_short": "calfews_src-data-CDEC_19052016.csv",
            "CDEC_1995_2023": "calfews_src-data-CDEC_1005_2023",
            "CDEC_1996_2024": "calfews_src-data-CDEC_1996_2024"
        },
        "synthetic": {}
    },
    
    "simulation_period_start": {
        "observations":{
            "CDEC": 1905, 
            "CDEC_wet_dry": 1996,
            "CDEC_short": 1996,
            "CDEC_1995_2023":1995,
            "CDEC_1996_2024":1996
            },
        "synthetic": {}
    },

    "simulation_period_end":{
        "observations":{
        "CDEC": 2016,
        "CDEC_wet_dry": 1998,
        "CDEC_short": 2024,
        "CDEC_1995_2023":2023,
        "CDEC_1996_2024":2024 
        },
        "synthetic": {}
    },
    "file_start":{
        "observations":{
        "CDEC": "10/1/1905",
        "CDEC_wet_dry": "10/1/1996",
        "CDEC_short": "10/1/1996",
        "CDEC_1995_2023":"10/1/1995",
        "CDEC_1996_2024":"10/1/1996"
        },
        "synthetic": {}
    },
    "file_end":{
    "observations":{
      "CDEC": "9/30/2016",
      "CDEC_wet_dry": "9/30/1998",
      "CDEC_short": "9/30/2024",
      "CDEC_1995_2023":"9/30/2023",
      "CDEC_1996_2024":"9/30/2024"
      },
         "synthetic": {}
    },
      "has_full_inputs":{
    "observations":{
      "CDEC": False,
      "CDEC_wet_dry": False,
      "CDEC_short": True,
      "CDEC_1995_2023": True,
      "CDEC_1996_2024": True
      },
        "synthetic": {}
    },
     "has_leap":{
    "observations":{
      "CDEC":True,
      "CDEC_wet_dry": True,
      "CDEC_short": True,
      "CDEC_1995_2023":True,
      "CDEC_1996_2024":True
      },
                    "synthetic": {}
    },
      "inflow_unit":{
    "observations":{
      "CDEC": "af",
      "CDEC_wet_dry": "af",
      "CDEC_short": "af",
      "CDEC_1995_2023":"af",
      "CDEC_1996_2024":"af"
      },
    "synthetic": {}
    },
}

# Adding synthetic data for years 1996 to 2024 and samples from 1 to 100
for year in range(1996, 2025):
    for sample in range(1, 101):
        # Adding entries for 'synthetic' and 'export_series' with year and sample
        year_sample_key = f"{year}_{sample}"
        data["flow_input_source"]["synthetic"][year_sample_key] = f"calfews_src/data/MGHMM_synthetic/start_year/DailyQ_s_{year}_{sample-1}.csv"
        data["export_series"]["synthetic"][year_sample_key] = f"calfews_src-data-synth_{year}_{sample}"
        data["simulation_period_start"]["synthetic"][year_sample_key] = 2024
        data["simulation_period_end"]["synthetic"][year_sample_key] = 2027
        data["file_start"]["synthetic"][year_sample_key] = "10/1/2024"
        data["file_end"]["synthetic"][year_sample_key] = "9/30/2027"
        data["has_full_inputs"]["synthetic"][year_sample_key] = False
        data["has_leap"]["synthetic"][year_sample_key] = True
        data["inflow_unit"]["synthetic"][year_sample_key] = "af"



# Write the data to a JSON file
json_filename = "base_inflows_synthetic.json"
with open(json_filename, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"JSON data has been written to {json_filename}")


JSON data has been written to base_inflows_synthetic.json
